In [1]:
import pandas as pd
import pyspark
from pyspark.sql import SparkSession, types, functions as F

In [2]:
spark = SparkSession\
  .builder\
  .master("local[*]")\
  .appName('test')\
  .getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/03/11 22:17:36 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
df_green = spark.read.parquet('./data/pq/green/*/*')

In [4]:
df_green.createOrReplaceTempView('trip_data')

In [9]:
df_green_revenue = spark.sql("""
  SELECT 
    date_trunc("hour", lpep_pickup_datetime) AS hour,
    PULocationID AS zone,
    SUM(total_amount) AS amount,
    COUNT(*) AS row_count
  FROM trip_data
  WHERE lpep_pickup_datetime >= '2020-01-01 00:00:00'
  GROUP BY 1, 2
  ORDER BY 1, 2
""")

In [10]:
df_green_revenue.show()

+-------------------+----+------------------+---------+
|               hour|zone|            amount|row_count|
+-------------------+----+------------------+---------+
|2020-01-01 00:00:00|   7|1539.4599999999984|       90|
|2020-01-01 00:00:00|  17|390.06000000000006|       18|
|2020-01-01 00:00:00|  18|              15.6|        2|
|2020-01-01 00:00:00|  22|              31.6|        2|
|2020-01-01 00:00:00|  24|             175.2|        6|
|2020-01-01 00:00:00|  25|1061.9999999999995|       52|
|2020-01-01 00:00:00|  29|             122.6|        2|
|2020-01-01 00:00:00|  32|             137.9|        4|
|2020-01-01 00:00:00|  33|            634.54|       22|
|2020-01-01 00:00:00|  35|            259.92|       10|
|2020-01-01 00:00:00|  36|            590.68|       22|
|2020-01-01 00:00:00|  37|            351.34|       12|
|2020-01-01 00:00:00|  38|            197.58|        4|
|2020-01-01 00:00:00|  40|337.96000000000004|       16|
|2020-01-01 00:00:00|  41|2727.9200000000005|   

In [ ]:
df_green_revenue.write.parquet('data/report/revenue/green')

In [ ]:
spark.stop()